<a href="https://colab.research.google.com/github/mmmhdp/we_kan_do_it/blob/dev/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
  <h1 align="center"></h1>
  <h1 align="center">GPT2 c KAN слоями. Тех разбор</h1>
  <p align="center">
    <a href="https://github.com/mmmhdp">Мальцев Никита</a>
    ·
    <a href="https://github.com/anyhhope">Надежда Анисимова</a>
    ·
    <a href="https://github.com/TeamSyncher">Ярослав Исаков</a>
    ·
    <a href="https://github.com/staryarych">Ярослав Столбовский</a>
    ·
    <a href="https://github.com/Mariiiiiiiia">Мария Бородкина</a>
    
</p>
   <h3 align="center">
        <a href="https://github.com/mmmhdp/we_kan_do_it">GitHub</a> |
        <a href="https://www.loom.com/share/7a8caa786faf4b74bfa99fe040e9f549">Video Presentation</a> |
        <a href="">Raw Presentation</a>
   </h3>
  <div align="center"></div>
</p>

# Введение
краткое введение

## Постановка задачи
- Имплементировать KAN.
- Оценить рост эффективности KAN в сравнении с MLP на базовых задачах.
- Оценить рост эффективности работы GPT2 модели по ряду выбранных метрик, если будет произведен замена MLP слоя на KAN слой.

## Описание используемых данных
Тут могла быть ваша реклама

### Предобработка данных
Тут могла быть ваша реклама


### Токенизация
Тут могла быть ваша реклама


### Разбиение на выборки
Тут могла быть ваша реклама

## Используемые метрики
Тут могла быть ваша реклама

# KAN. Описание и имплементация

## Описание
Математическая справка?

## Имплементация

# KAN vs MLP в базовых задачах

# GPT2

In [53]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [54]:
# hyperparameters
BATCH_SIZE = 8
BLOCK_SIZE = 32
MAX_ITERS = 1
EVAL_INTERVAL = 1
LEARNING_RATE = 3e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EVAL_ITERS = 1
N_EMBD = 128
N_HEAD = 6 # D = 384 // 6
N_LAYER = 6
DROPOUT = 0.2
# ------------

torch.manual_seed(42)

In [55]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


In [56]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

In [57]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [58]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(N_EMBD, head_size, bias=False)
        self.query = nn.Linear(N_EMBD, head_size, bias=False)
        self.value = nn.Linear(N_EMBD, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))

        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        # input of size (B, T, C)
        # output of size (B, T, D)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,D)
        q = self.query(x) # (B,T,D)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, D) @ (B, D, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        v = self.value(x) # (B,T,D)
        out = wei @ v # (B, T, T) @ (B, T, D) -> (B, T, D)
        return out

In [59]:
class NaiveMultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [60]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()

        self.D = head_size
        self.H = num_heads
        self.key = nn.Linear(N_EMBD, head_size * num_heads, bias=False)
        self.query = nn.Linear(N_EMBD, head_size * num_heads, bias=False)
        self.value = nn.Linear(N_EMBD, head_size * num_heads, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))
        self.proj = nn.Linear(head_size * num_heads, N_EMBD)

        self.dropout = nn.Dropout(DROPOUT)


    def forward(self, x):
        
        B,T,C = x.shape
        H = self.H
        D = self.D

        k = self.key(x).view(B, T, H, D)
        q = self.query(x).view(B, T, H, D)
        v = self.value(x).view(B, T, H, D)

        k = k.permute(0, 2, 3, 1) 
        q = q.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)

        wei = q @ k * k.shape[-1]**-0.5

        wei = wei.masked_fill_(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        out = wei @ v
        out = out.permute(0, 2, 1, 3).reshape(B, T, -1)

        out = self.dropout(self.proj(out))
        return x


In [61]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(DROPOUT),
        )

    def forward(self, x):
        return self.net(x)


In [62]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [63]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(VOCAB_SIZE, N_EMBD)
        self.position_embedding_table = nn.Embedding(BLOCK_SIZE, N_EMBD)
        self.blocks = nn.Sequential(*[Block(N_EMBD, n_head=N_HEAD) for _ in range(N_LAYER)])
        self.ln_f = nn.LayerNorm(N_EMBD)
        self.lm_head = nn.Linear(N_EMBD, VOCAB_SIZE)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=DEVICE)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [64]:
model = GPTLanguageModel()
m = model.to(DEVICE)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

1.199928 M parameters


In [65]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

    if iter % EVAL_INTERVAL == 0 or iter == MAX_ITERS - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.0531, val loss 4.0127


In [66]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


;» я»гОСиВ:К-Х«СИ—ьчТ-ыыЧдС!Вхс,гтжнС?эХИцА—зсИчы..юнчэсвеТ,чзшХухг-дОНПиуюдХПсСевНунжИ?дьлтёыбс;АэХХгХш,Мрозю:;ижпЧц;вбК ооуь
и;т»ТйпЧёкч»НПеАинтэНку«д-кед-яцбЛц;ЧНзйя?ц«дя«яЛОм,Ии«.ЛОов
я«ч»рг?к—оышмВТ:Олёп?.с;чвсВ».учКхцНг?оэУЧНп;э-ТйоТ-СлоеУКО.у.с: .—рди—жунусА юз ,:хю.уйУ»ы;тцюувдэс?свэдт
зяАИмз.,ЛеКЛшАмМдтс»яп,ьНввН;у—х«гКи?ч»НВЛа»Пя
?,,д пХвЧвпхПйАйы»;уёХУНтМ
ьМрваК
ипЛюьугиКПОм Л:УКуКнМмэвСш—оИ ыСч»ьз;б—лы,;КЛ-и,ТвАп:пи»Нп Птти уннОыеУяух;лёжьч
юыкцчКЛ р—кмоупсолшнаВКы
!х,йи—эитоля-цО;юТ


# KAN vs MLP в контесте GPT2